In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
import optuna
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegressionCV
import joblib
import shap

/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load both datasets
original = pd.read_csv("PCOS_data.csv")
new = pd.read_csv("pcos_dataset.csv")

# Set max columns to show is unlimited
pd.set_option('display.max_columns', None)

In [3]:
def preprocess(df):
    df = df.copy()  # avoid SettingWithCopyWarning

    # 1. Clean column names
    df.columns = df.columns.str.strip() \
                           .str.replace(' ', '_') \
                           .str.replace('(', '') \
                           .str.replace(')', '') \
                           .str.replace('.', '') \
                           .str.replace('-', '_') \
                           .str.replace('/', '_')
    df.rename(columns={'II____beta_HCGmIU_mL': 'II_beta_HCG'}, inplace=True)

    # 2. Drop irrelevant columns
    df.drop(columns=['Sl_No', 'Patient_File_No'], inplace=True, errors='ignore')
    df = df.loc[:, ~df.columns.str.startswith('Unnamed')]

    # 3. Merge Age columns
    if 'Age' not in df.columns and 'Age_yrs' in df.columns:
        df.rename(columns={'Age_yrs': 'Age'}, inplace=True)
    elif 'Age' in df.columns and 'Age_yrs' in df.columns:
        df['Age'] = df['Age'].fillna(df['Age_yrs'])
        df.drop(columns=['Age_yrs'], inplace=True)

    # 4. Merge PCOS diagnosis columns
    if 'PCOS_Diagnosis' in df.columns:
        df.rename(columns={'PCOS_Diagnosis': 'PCOS_Y_N'}, inplace=True)

    # 5. Handle missing values
    if 'Marraige_Status_Yrs' in df.columns:
        df.loc[:, 'Marraige_Status_Yrs'] = df['Marraige_Status_Yrs'].fillna(df['Marraige_Status_Yrs'].median())

    if 'Fast_food_Y_N' not in df.columns and 'Fast_food_YN' in df.columns:
        df.rename(columns={'Fast_food_YN': 'Fast_food_Y_N'}, inplace=True)
    if 'Fast_food_Y_N' in df.columns:
        df.loc[:, 'Fast_food_Y_N'] = df['Fast_food_Y_N'].fillna(df['Fast_food_Y_N'].mode()[0])

    # 6. Convert to numeric and fill missing values
    if 'II_beta_HCG' not in df.columns and 'II_beta_HCGmIU_mL' in df.columns:
        df.rename(columns={'II_beta_HCGmIU_mL': 'II_beta_HCG'}, inplace=True)
    if 'II_beta_HCG' in df.columns:
        df.loc[:, 'II_beta_HCG'] = pd.to_numeric(df['II_beta_HCG'], errors='coerce')
        df['II_beta_HCG'] = df['II_beta_HCG'].astype(float)
        df.loc[:, 'II_beta_HCG'] = df['II_beta_HCG'].fillna(df['II_beta_HCG'].median())

    if 'AMHng_mL' in df.columns:
        df.loc[:, 'AMHng_mL'] = pd.to_numeric(df['AMHng_mL'], errors='coerce')
        df['AMHng_mL'] = df['AMHng_mL'].astype(float)
        df.loc[:, 'AMHng_mL'] = df['AMHng_mL'].fillna(df['AMHng_mL'].median())

    return df

In [4]:
# Apply preprocessing
original_clean = preprocess(original)
new_clean = preprocess(new)

# Ensure consistent columns across both
all_columns = list(set(original_clean.columns).union(set(new_clean.columns)))

# Align both dataframes to same columns, fill missing with NaN
original_aligned = original_clean.reindex(columns=all_columns)
new_aligned = new_clean.reindex(columns=all_columns)

# Concatenate datasets
combined_df = pd.concat([original_aligned, new_aligned], ignore_index=True)

In [5]:
# Separate features and target
# X = combined_df.drop(columns=['PCOS_Y_N'])
X = combined_df.drop(columns=['PCOS_Y_N'])
y = combined_df['PCOS_Y_N']

# Initialize KNNImputer (e.g., with 5 nearest neighbors)
knn_imputer = KNNImputer(n_neighbors=5)

X = pd.DataFrame(knn_imputer.fit_transform(X), columns=X.columns)

# Sanitize feature names
X.columns = [str(col).replace(' ', '_')
                        .replace('"', '')
                        .replace("'", '')
                        .replace('[', '')
                        .replace(']', '')
                        .replace('{', '')
                        .replace('}', '')
                        .replace(':', '')
                        .replace(',', '')
                        for col in X.columns]

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

In [7]:
# Step 3: Train model on full data
model = RandomForestClassifier(n_estimators=100, random_state=8)
model.fit(X, y)

# Step 4: Run SHAP on full data to find top features
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

shap_values_class1 = shap_values[:, :, 1]  # shape: (1541 samples, 44 features)
mean_abs_shap = np.abs(shap_values_class1).mean(axis=0)  # shape: (44,)

# Step 5: Create DataFrame of feature importances
shap_df = pd.DataFrame({
    'feature': X.columns,
    'importance': mean_abs_shap
}).sort_values(by='importance', ascending=False)

# Step 6: Select top N features
top_n = 20
top_features = shap_df['feature'].head(top_n).tolist()

In [8]:
top_features

['Menstrual_Irregularity',
 'BMI',
 'Testosterone_Levelng_dL',
 'Follicle_No_R',
 'Weight_gainY_N',
 'Skin_darkening_Y_N',
 'Antral_Follicle_Count',
 'Weight_Kg',
 'hair_growthY_N',
 'Follicle_No_L',
 'CycleR_I',
 'Waistinch',
 'Cycle_lengthdays',
 'Fast_food_Y_N',
 'AMHng_mL',
 'PimplesY_N',
 'Hipinch',
 'Age',
 'FSHmIU_mL',
 'FSH_LH']

In [9]:
X = X[top_features]
X

,Menstrual_Irregularity,BMI,Testosterone_Levelng_dL,Follicle_No_R,Weight_gainY_N,Skin_darkening_Y_N,Antral_Follicle_Count,Weight_Kg,hair_growthY_N,Follicle_No_L,CycleR_I,Waistinch,Cycle_lengthdays,Fast_food_Y_N,AMHng_mL,PimplesY_N,Hipinch,Age,FSHmIU_mL,FSH_LH
0,0.656240,-1.339000,0.920567,-1.122376,-1.087347,-0.985781,-0.416841,-1.585428,-0.946511,-1.004611,-0.960414,-1.507280,0.114785,1.308089,-0.908724,-1.396720,-0.862541,-0.484837,-0.019471,-0.112564
1,-0.744405,-0.159445,-0.683756,-0.495512,-1.087347,-0.985781,-0.551245,0.188490,-0.946511,-1.004611,-0.960414,-0.850477,0.114785,-1.447194,-1.036757,-1.396720,-0.261702,0.577375,-0.028442,-0.009494
2,1.123121,-0.075192,0.412176,2.638810,-1.087347,-0.985781,0.154377,0.518926,-0.946511,2.295832,-0.960414,0.463129,0.114785,1.308089,0.172444,1.419749,0.339137,0.179046,-0.037192,-0.006153
3,0.656240,0.851601,-0.414846,-1.435808,-1.087347,-0.985781,-0.215234,0.188490,-0.946511,-1.334656,-0.960414,0.463129,0.114785,-1.447194,-1.110258,-1.396720,0.939976,0.710152,-0.018662,-0.080178
4,-0.277523,-1.170492,-0.029241,-0.808944,-1.087347,-0.985781,-0.248835,-0.941948,-0.946511,-1.004611,-0.960414,-1.507280,0.114785,-1.447194,-0.863675,-1.396720,-0.562122,-0.883166,-0.048663,-0.054475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,1.123121,-1.528571,1.824711,-0.370139,-1.087347,0.202399,0.927202,-1.498471,-0.311797,-0.674567,-0.960414,-2.361125,0.114785,-0.896137,-0.138155,-1.396720,-2.124303,0.311822,-0.034339,-0.079356
1537,1.123121,0.683094,-1.584855,-0.182079,0.455564,-0.391691,-1.760884,0.197186,-0.311797,0.249557,-0.357932,0.594490,-0.080236,-1.447194,-0.602393,-0.270132,0.459304,1.772363,-0.047825,-0.001269
1538,-1.211286,0.556713,-1.386979,-0.370139,-0.058739,-0.985781,1.767229,0.569361,-0.311797,-0.146496,-0.357932,0.463129,0.504828,-0.345081,-0.146216,-0.833426,0.519388,0.710152,-0.046134,-0.072673
1539,-1.211286,0.346078,1.819637,-0.808944,1.484172,0.796490,-1.424873,0.649361,-0.311797,-0.344523,0.847033,1.185613,0.114785,1.308089,0.042040,-0.270132,0.819808,1.241257,-0.038751,-0.127420


In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y, shuffle=True, random_state=8)

In [11]:
# Define the objective function for Random Forest
def objective(trial):
    # Define hyperparameters for Random Forest
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "max_depth": trial.suggest_int("max_depth", 3, 12, log=True),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "random_state": 42  # Set a fixed random seed for reproducibility
    }

    # Train the Random Forest model with the suggested hyperparameters
    model = RandomForestClassifier(**param)
    model.fit(x_train, y_train)

    # Predict on the validation set
    preds = model.predict(x_test)

    # Calculate accuracy
    acc = accuracy_score(y_test, preds)

    # Return the negative accuracy because Optuna minimizes the objective
    return 1.0 - acc  # Minimize the error (lower is better)

In [12]:
study = optuna.create_study(direction='minimize')  # Because we return 1 - accuracy
study.optimize(objective, n_trials=50)  # Try 50 combinations (or more!)

print("Best hyperparameters:")
print(study.best_params)

print("Best accuracy:")
print(1 - study.best_value)

[I 2025-05-31 20:29:36,769] A new study created in memory with name: no-name-cabb2c33-2a95-45d4-9b47-2500bf267725
[I 2025-05-31 20:29:36,956] Trial 0 finished with value: 0.02155172413793105 and parameters: {'n_estimators': 160, 'max_depth': 10, 'min_samples_split': 4, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.02155172413793105.
[I 2025-05-31 20:29:37,494] Trial 1 finished with value: 0.030172413793103425 and parameters: {'n_estimators': 160, 'max_depth': 11, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': None}. Best is trial 0 with value: 0.02155172413793105.
[I 2025-05-31 20:29:37,573] Trial 2 finished with value: 0.03448275862068961 and parameters: {'n_estimators': 68, 'max_depth': 9, 'min_samples_split': 7, 'min_samples_leaf': 7, 'max_features': 'log2'}. Best is trial 0 with value: 0.02155172413793105.
[I 2025-05-31 20:29:37,720] Trial 3 finished with value: 0.1594827586206896 and parameters: {'n_estimators': 194, 'max_depth': 3,

Best hyperparameters:
{'n_estimators': 127, 'max_depth': 10, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 'sqrt'}
Best accuracy:
0.9827586206896551


In [13]:
# Get the data
best_model = RandomForestClassifier(**study.best_params)
best_model.fit(x_train, y_train)

# Predict
preds = best_model.predict(x_test)

# Compute final scores
acc = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='binary')
recall = recall_score(y_test, preds, average='binary')
f1 = f1_score(y_test, preds, average='binary')

print("\nFinal evaluation on test set:")
print(f"Accuracy: {acc * 100:.2f}")
print(f"Precision: {precision * 100:.2f}")
print(f"Recall: {recall * 100:.2f}")
print(f"F1 Score: {f1 * 100:.2f}")



Final evaluation on test set:
Accuracy: 97.41
Precision: 98.11
Recall: 91.23
F1 Score: 94.55
